# MODEL Deployment: Inference on New Data

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

region = boto3.Session().region_name

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")

sess = sagemaker.Session()

# project bucket
bucket_name = "aai-540-group4"

# image source and lst files
images_prefix = "datasets/cct_resized"
s3_images_location = f"s3://{bucket_name}/{images_prefix}/"
s3_images_eval = f"s3://{bucket_name}/{images_prefix}/val/"
#train_lst_key = "
#val_lst_key = "

# specifiy output location of training data and model
output_prefix = "sg-ic-transfer-learning"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::324183265896:role/service-role/AmazonSageMaker-ExecutionRole-20250604T045982


In [2]:
from sagemaker.model import Model
from sagemaker import image_uris

# base image uri
training_image = image_uris.retrieve(
    framework = "image-classification", region = sess.boto_region_name, version="latest"
)

model = Model(
    image_uri=training_image,  # e.g., the URI used for training
    model_data='s3://aai-540-group4/sg-ic-transfer-learning/output/image-classification-2025-06-12-05-31-48-116/output/model.tar.gz',  # S3 path to your model artifact
    role=role
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge'  # or another suitable instance type
)

-----------!

In [4]:
import json
# load index to category label 
with open('./label_to_idx.json', 'r') as file:  # Replace with your file name if different
    label_to_idx = json.load(file)

# invert: idx to label
idx_to_label = {v: k for k, v in label_to_idx.items()}


In [6]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

# Replace with your actual endpoint name 
endpoint_name = 'image-classification-2025-06-12-12-23-14-124' # update code later, harcoded for now


In [7]:

# Create a Predictor object for the existing endpoint
# Set up the predictor with the appropriate serializer for raw image bytes
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=IdentitySerializer(content_type='application/x-image'),  # For .jpg/.png images
    deserializer=JSONDeserializer()  # Assumes the model returns JSON
)



In [8]:
import numpy as np

In [9]:
# try on one image

# Now you can use predictor.predict() as usual
img_path = '5858bf6d-23d2-11e8-a6a3-ec086b02610b_0.jpg'
with open(img_path, 'rb') as f:
    payload = f.read()

result = predictor.predict(payload)

print(idx_to_label[np.argmax(result)])

dog


In [10]:
import os
from IPython.display import clear_output
import time
import pandas as pd


preds_df = pd.DataFrame(columns=['filename', 'pred_label'])
init_time = time.time()

paginator = s3_client.get_paginator('list_objects_v2')

# List objects in S3 bucket
i= 0
for page in paginator.paginate(Bucket=bucket_name, Prefix='datasets/cct_resized/val'):
    for obj in page.get('Contents', []):
        # Download file
        data = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])['Body'].read()
        
        # Send to endpoint
        result = predictor.predict(data)
        #print(f"i: {i}")
        #print(obj['Key'])
        filename = os.path.basename(obj['Key'])
        pred_label = idx_to_label[np.argmax(result)]
        
        #print(os.path.basename(obj['Key']))  # Extracts 'image1.jpg'
        #print(result)
        #print(np.argmax(result))
        #print(f"label: {idx_to_label[np.argmax(result)]}")

        #pred = {
        #   "filename" : filename,
            #"pred_probs" : result,
            #"label_idx_max" : np.argmax(result),
        #    "pred_label" : idx_to_label[np.argmax(result)]
        #}
        preds_df.loc[len(preds_df)] = [filename, pred_label]
        #preds.append(pred)
        
        i = i+1

        if(i%1000 == 0): 
            now_time = time.time()
            clear_output(wait=True)
            print(f"Val images predicted so far: {i}, Time Elapsed: {now_time - init_time:.4f} seconds")
            time.sleep(0.1)

            # Save predictions to S3
            s3_client.put_object(
                Bucket=bucket_name,
                #Prefix=output_prefix,
                Key=f"{output_prefix}/predictions/update-preds-{i}.json",
                Body=preds_df.to_json(orient='records')
            )

        #if(i%20 == 0): break

    #break
        
             
        # Save predictions to S3
        #s3.put_object(
        #    Bucket=bucket_name,
        #    Prefix='datasets/cct_resized/val',
        #    Key=f"{obj['Key']}.json",
        #    Body=json.dumps(response)
        #)




Val images predicted so far: 25000, Time Elapsed: 1472.1035 seconds


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:20                                                                                   │
│                                                                                                  │
│   17 │   │   data = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])['Body'].read()      │
│   18 │   │                                                                                       │
│   19 │   │   # Send to endpoint                                                                  │
│ ❱ 20 │   │   result = predictor.predict(data)                                                    │
│   21 │   │   #print(f"i: {i}")                                                                   │
│   22 │   │   #print(obj['Key'])                                                                  │
│   23 │   │   filename = os.path.basename(obj['Key'])                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/base_predictor.py:212 in predict               │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                     

In [11]:
i

25751

In [12]:
# save last update
s3_client.put_object(
    Bucket=bucket_name,
    #Prefix=output_prefix,
    Key=f"{output_prefix}/predictions/update-preds-{i}.json",
    Body=preds_df.to_json(orient='records')
)

{'ResponseMetadata': {'RequestId': 'Q6W3W0BXMXEFTG2E',
  'HostId': 'kB4AbDwdoXkbRz9Z8BRgRNCkF5ul7c69noF/RuWh8Cc7/Ua8p7zhksx/yyJNNZjrJCu2kyYPtzd1kn7yD5fLAaulG2e3F9DBOydLhnT1Jgc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'kB4AbDwdoXkbRz9Z8BRgRNCkF5ul7c69noF/RuWh8Cc7/Ua8p7zhksx/yyJNNZjrJCu2kyYPtzd1kn7yD5fLAaulG2e3F9DBOydLhnT1Jgc=',
   'x-amz-request-id': 'Q6W3W0BXMXEFTG2E',
   'date': 'Thu, 12 Jun 2025 13:06:55 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5c4e8de59388f9647961df06bac5e8f5"',
   'x-amz-checksum-crc32': '029sBw==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"5c4e8de59388f9647961df06bac5e8f5"',
 'ChecksumCRC32': '029sBw==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}